In [1]:
using ArgParse
using CSV
using DataFrames
using Glob
using Gurobi
using JuMP
using Printf
using StatsBase

include(joinpath(@__DIR__, "..", "src", "ItineraryPlanning.jl"))
using .ItineraryPlanning: DayBefore, benders, direct_model_solve, ParetoArguments

# Reproducing results 
This notebook shows how to reproduce results from "Activated Benders Decomposition for Paratransit Itinerary Planning" by KC, AJ, VV in INFORMS Journal of Computing.

We will load in raw trip data; we will format it into SIPPAR model inputs; and we will solve the model with a variety of methods:
* Direct implementation
  - Integer recourse 
* Benders decomposition implementation 
  - Activated vs traditional
  - Pareto-optimal cuts 
  - (Activated) locally Pareto-optimal cuts

From there, readers will be able to replicate analysis associated with results of different formats.

In [2]:
# create a directory to store model outcomes
output_dir = joinpath(@__DIR__, "..", "other_results")
mkpath(output_dir)  

"/Users/kaylacummings/Desktop/2023.0311/notebooks/../other_results"

In [3]:
# load the model inputs
data_dir = joinpath(@__DIR__, "..", "data", "case_study")
num_scenarios = 1
num_itin = 10000
W = 20
f = 0.0
labor_min_planned = 20/60
labor_min_adhoc = 30/60
delay_penalty = 100.0
D = DayBefore(data_dir, num_scenarios, num_itin, W, f, labor_min_planned, labor_min_adhoc, delay_penalty);

In [4]:
# configure preferences for which model to solve
relax = true # relaxed recourse 
swap = true # whether to allow swap recourse
activated = true # whether to solve with activated benders decomposition

true

### Solve directly

Solve the model without any decomposition methods.

In [5]:
st, obj, gap, x_val, ss =  direct_model_solve(D, swap, !relax)

Set parameter TimeLimit to value 3600
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i9-8950HK CPU @ 2.90GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 10112 rows, 170427 columns and 441738 nonzeros
Model fingerprint: 0x7a6dbefa
Variable types: 0 continuous, 170427 integer (170427 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-15, 4e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 231.0333333
Presolve removed 62 rows and 40904 columns
Presolve time: 0.93s
Presolved: 10050 rows, 129523 columns, 302127 nonzeros
Found heuristic solution: objective 214.1333333
Variable types: 0 continuous, 129523 integer (129523 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.06s

Solved with primal simplex

Root relaxation: objecti

(7.296022928, 143.63333333333327, 0.0, 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, ["src-5-sink", "src-16-sink", "src-16-5-sink", "src-2-sink", "src-26-sink", "src-22-sink", "src-28-sink", "src-19-sink", "src-4-sink", "src-25-sink"  …  "src-29-5-22-20-28-19-12-14-sink", "src-29-16-5-22-20-28-19-12-14-sink", "src-29-8-5-22-20-28-19-12-14-sink", "src-29-8-27-20-28-19-12-14-sink", "src-29-6-27-22-20-28-19-12-14-sink", "src-29-8-21-22-20-28-19-12-14-sink", "src-29-8-22-20-28-19-12-14-sink", "src-29-27-22-20-28-19-12-14-sink", "src-2-16-5-22-20-28-19-12-26-sink", "src-2-16-5-22-20-28-19-12-14-sink"]
And data, a 10000-element Vector{Float64}:
  0.0
  0.0
  1.0
  1.0
 -0.0
 -0.0
 -0.0
  1.0
  1.0
  1.0
  ⋮
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0, -217.89999999999998)

### Solve with traditional Benders

Solve with BD, without any algorithmic enhancements.

In [6]:
lb, ub, mb, mp_times, sp_parallel_times, sp_serial_times, x, ss, ss_vals, speedy_rounds = benders(!activated, D, swap)

Set parameter TimeLimit to value 1800
Iteration 1...
Set parameter TimeLimit to value 1800
Scenario cancel 21,29 dns ...
Set parameter DualReductions to value 0
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 6000
Set parameter TimeLimit to value 6000
MILP: 186.10 - MIP: 186.10 - Bound: -1000000.00
Iteration 2...
Scenario cancel 21,29 dns ...
Set parameter DualReductions to value 0
Set parameter TimeLimit to value 600
MILP: 186.10 - MIP: 186.10 - Bound: 229.12
Iteration 3...
Scenario cancel 21,29 dns ...
Set parameter DualReductions to value 0
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 6000
Set parameter TimeLimit to value 6000
MILP: 144.57 - MIP: 144.57 - Bound: 137.88
Iteration 4...
Scenario cancel 21,29 dns ...
Set parameter DualReductions to value 0
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 6000
Set parameter TimeLimit to value 6000
MILP: 143.63 - MIP: 143.63 - Bound: 143.63
Set parameter TimeLimit to valu

(Any[-1.0e6, 229.11666666666667, 137.8833333333333, 143.63333333333324], Any[186.09999999999994, 186.09999999999994, 144.56666666666666, 143.63333333333327], Any[186.09999999999994, 186.09999999999994, 144.56666666666672, 143.6333333333333], Any[0.325314491, 0.065256481, 0.133386257, 0.232424877], Any[1.216377457, 1.020804403, 1.02890469, 1.115466311], Any[1.216377457, 1.020804403, 1.02890469, 1.115466311], 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, ["src-5-sink", "src-16-sink", "src-16-5-sink", "src-2-sink", "src-26-sink", "src-22-sink", "src-28-sink", "src-19-sink", "src-4-sink", "src-25-sink"  …  "src-29-5-22-20-28-19-12-14-sink", "src-29-16-5-22-20-28-19-12-14-sink", "src-29-8-5-22-20-28-19-12-14-sink", "src-29-8-27-20-28-19-12-14-sink", "src-29-6-27-22-20-28-19-12-14-sink", "src-29-8-21-22-20-28-19-12-14-sink", "src-29-8-22-20-28-19-12-14-sink", "src-29-27-22-20-28-19-12-14-sink", "src-2-16-5-22-20-28-19-12-26-sink", "src-2-16-5-22-20-28-19-12-14

### Solve with activated Benders

Solve with ABD.

In [7]:
lb, ub, mb, mp_times, sp_parallel_times, sp_serial_times, x, ss, ss_vals, speedy_rounds = benders(activated, D, swap)

Set parameter TimeLimit to value 1800
Iteration 1...
Set parameter TimeLimit to value 1800
Scenario cancel 21,29 dns ...
Set parameter DualReductions to value 0
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 6000
Set parameter TimeLimit to value 6000
MILP: 186.10 - MIP: 186.10 - Bound: -1000000.00
Iteration 2...
Scenario cancel 21,29 dns ...
Set parameter DualReductions to value 0
Set parameter TimeLimit to value 600
MILP: 186.10 - MIP: 186.10 - Bound: 229.12
Iteration 3...
Scenario cancel 21,29 dns ...
Set parameter DualReductions to value 0
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 6000
Set parameter TimeLimit to value 6000
MILP: 146.03 - MIP: 146.03 - Bound: 51.95
Iteration 4...
Scenario cancel 21,29 dns ...
Set parameter DualReductions to value 0
Set parameter TimeLimit to value 600
MILP: 146.03 - MIP: 146.03 - Bound: 63.93
Iteration 5...
Scenario cancel 21,29 dns ...
Set parameter DualReductions to value 0
Set parameter TimeLimit 

(Any[-1.0e6, 229.11666666666667, 51.949999999999875, 63.93333333333325, 90.18333333333322, 92.13333333333321, 106.63333333333335, 118.20000000000002, 122.81666666666658, 125.14999999999992  …  142.88333333333327, 142.89999999999998, 143.03333333333325, 143.16666666666669, 143.30000000000007, 143.36666666666665, 143.36666666666665, 143.36666666666662, 143.56666666666663, 143.6333333333333], Any[186.09999999999994, 186.09999999999994, 146.0333333333333, 146.0333333333333, 146.0333333333333, 146.0333333333333, 146.0333333333333, 146.0333333333333, 144.56666666666666, 144.56666666666666  …  143.63333333333327, 143.63333333333327, 143.63333333333327, 143.63333333333327, 143.63333333333327, 143.63333333333327, 143.63333333333327, 143.63333333333327, 143.63333333333327, 143.63333333333327], Any[186.09999999999994, 186.09999999999994, 146.03333333333342, 146.03333333333342, 146.03333333333342, 146.03333333333342, 146.03333333333342, 146.03333333333342, 144.56666666666672, 144.56666666666672  …

### Solve with PO cuts

There are three options:
* regular Pareto optimal cuts 
* locally Pareto optimal cuts 
* activated LPO cuts

In [8]:
pareto = true
pareto_n = 10 # number of feasible solutions for core point
dynamic_n = 25 # rolling window length for activated core point
pareto_activated = true
localpareto = true
regular_po = ParetoArguments(pareto, !localpareto, !pareto_activated, pareto_n, dynamic_n)
local_po = ParetoArguments(!pareto, localpareto, !pareto_activated, pareto_n, dynamic_n)
act_local_po = ParetoArguments(!pareto, localpareto, pareto_activated, pareto_n, dynamic_n);

In [9]:
# ABD + regular PO 
lb, ub, mb, mp_times, sp_parallel_times, sp_serial_times, x, ss, ss_vals, speedy_rounds = benders(activated, D, swap, regular_po)

Set parameter TimeLimit to value 1800
Set parameter TimeLimit to value 1800
Set parameter TimeLimit to value 1800
Iteration 1...
Set parameter TimeLimit to value 1800
Scenario cancel 21,29 dns ...
Set parameter DualReductions to value 0
Set parameter TimeLimit to value 600
Set parameter DualReductions to value 0
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 6000
Set parameter TimeLimit to value 6000
MILP: 186.10 - MIP: 186.10 - Bound: -1000000.00
Set parameter TimeLimit to value 1800
Set parameter TimeLimit to value 1800
Iteration 2...
Scenario cancel 21,29 dns ...
Set parameter DualReductions to value 0
Set parameter TimeLimit to value 600
Set parameter DualReductions to value 0
Set parameter TimeLimit to value 600
MILP: 186.10 - MIP: 186.10 - Bound: 229.12
Set parameter TimeLimit to value 1800
Set parameter TimeLimit to value 1800
Iteration 3...
Scenario cancel 21,29 dns ...
Set parameter DualReductions to value 0
Set parameter TimeLimit to value 600
Set param

(Any[-1.0e6, 229.11666666666667, 143.63333333333327], Any[186.09999999999994, 186.09999999999994, 143.63333333333333], Any[186.09999999999994, 186.09999999999994, 143.63333333333333], Any[1.1462538820000001, 0.577286803, 0.210860256], Any[0.7020394040000001, 0.7082169850000001, 0.718937148], Any[0.7020394040000001, 0.7082169850000001, 0.718937148], 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, ["src-5-sink", "src-16-sink", "src-16-5-sink", "src-2-sink", "src-26-sink", "src-22-sink", "src-28-sink", "src-19-sink", "src-4-sink", "src-25-sink"  …  "src-29-5-22-20-28-19-12-14-sink", "src-29-16-5-22-20-28-19-12-14-sink", "src-29-8-5-22-20-28-19-12-14-sink", "src-29-8-27-20-28-19-12-14-sink", "src-29-6-27-22-20-28-19-12-14-sink", "src-29-8-21-22-20-28-19-12-14-sink", "src-29-8-22-20-28-19-12-14-sink", "src-29-27-22-20-28-19-12-14-sink", "src-2-16-5-22-20-28-19-12-26-sink", "src-2-16-5-22-20-28-19-12-14-sink"]
And data, a 10000-element Vector{Float64}:
  0.0
  0

In [10]:
# ABD + LPO 
lb, ub, mb, mp_times, sp_parallel_times, sp_serial_times, x, ss, ss_vals, speedy_rounds = benders(activated, D, swap, local_po)

Set parameter TimeLimit to value 1800
Set parameter TimeLimit to value 1800
Set parameter TimeLimit to value 1800
Set parameter TimeLimit to value 1800
Set parameter TimeLimit to value 1800
Iteration 1...
Set parameter TimeLimit to value 1800
Scenario 

┌ Info: Terminating local branching prematurely.
│   n = 10
│   j = 4
└ @ Main.ItineraryPlanning /Users/kaylacummings/Desktop/2023.0311/src/benders.jl:605


cancel 21,29 dns ...
Set parameter DualReductions to value 0
Set parameter TimeLimit to value 600
Set parameter DualReductions to value 0
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 6000
Set parameter TimeLimit to value 6000
MILP: 186.10 - MIP: 186.10 - Bound: -1000000.00
Set parameter TimeLimit to value 1800
Set parameter TimeLimit to value 1800
Iteration 2...
Scenario cancel 21,29 dns ...
Set parameter DualReductions to value 0
Set parameter TimeLimit to value 600
Set parameter DualReductions to value 0
Set parameter TimeLimit to value 600
MILP: 186.10 - MIP: 186.10 - Bound: 229.12
Set parameter TimeLimit to value 1800
Set parameter TimeLimit to value 1800
Iteration 3...
Scenario cancel 21,29 dns ...
Set parameter DualReductions to value 0
Set parameter TimeLimit to value 600
Set parameter DualReductions to value 0
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 6000
Set parameter TimeLimit to value 6000
MILP: 144.57 - MIP: 144.57 - Bou

(Any[-1.0e6, 229.11666666666667, 142.28333333333333, 143.63333333333324], Any[186.09999999999994, 186.09999999999994, 144.56666666666666, 143.63333333333327], Any[186.09999999999994, 186.09999999999994, 144.56666666666672, 143.6333333333333], Any[0.089014896, 0.07841370299999999, 0.146472888, 0.234749866], Any[0.941212726, 0.8640980229999999, 0.990022512, 1.0461924820000001], Any[0.941212726, 0.8640980229999999, 0.990022512, 1.0461924820000001], 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, ["src-5-sink", "src-16-sink", "src-16-5-sink", "src-2-sink", "src-26-sink", "src-22-sink", "src-28-sink", "src-19-sink", "src-4-sink", "src-25-sink"  …  "src-29-5-22-20-28-19-12-14-sink", "src-29-16-5-22-20-28-19-12-14-sink", "src-29-8-5-22-20-28-19-12-14-sink", "src-29-8-27-20-28-19-12-14-sink", "src-29-6-27-22-20-28-19-12-14-sink", "src-29-8-21-22-20-28-19-12-14-sink", "src-29-8-22-20-28-19-12-14-sink", "src-29-27-22-20-28-19-12-14-sink", "src-2-16-5-22-20-28-19-12-

In [11]:
# ABD + ALPO 
lb, ub, mb, mp_times, sp_parallel_times, sp_serial_times, x, ss, ss_vals, speedy_rounds = benders(activated, D, swap, act_local_po)

Set parameter TimeLimit to value 1800
Set parameter TimeLimit to value 1800
Set parameter TimeLimit to value 1800
Set parameter TimeLimit to value 1800
Set parameter TimeLimit to value 1800
Iteration 1...
Set parameter TimeLimit to value 1800
Scenario 

┌ Info: Terminating local branching prematurely.
│   n = 10
│   j = 4
└ @ Main.ItineraryPlanning /Users/kaylacummings/Desktop/2023.0311/src/benders.jl:605


cancel 21,29 dns ...
Set parameter DualReductions to value 0
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 6000
Set parameter TimeLimit to value 6000
MILP: 186.10 - MIP: 186.10 - Bound: -1000000.00
Set parameter TimeLimit to value 1800
Set parameter TimeLimit to value 1800
Iteration 2...
Scenario cancel 21,29 dns ...
Set parameter DualReductions to value 0
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 600
MILP: 186.10 - MIP: 186.10 - Bound: 229.12
Set parameter TimeLimit to value 1800
Set parameter TimeLimit to value 1800
Iteration 3...
Scenario cancel 21,29 dns ...
Set parameter DualReductions to value 0
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 6000
Set parameter TimeLimit to value 6000
MILP: 151.85 - MIP: 151.85 - Bound: 108.2

(Any[-1.0e6, 229.11666666666667, 108.28333333333336, 139.88333333333324, 142.63333173092593, 143.45000000000002, 143.6333333333333], Any[186.09999999999994, 186.09999999999994, 151.8499999999999, 144.56666666666666, 143.63332622800326, 143.63332622800326, 143.63332622800326], Any[186.09999999999994, 186.09999999999994, 151.84999999999997, 144.56666666666672, 143.63332622800328, 143.63332622800328, 143.63332622800328], Any[0.097598146, 0.08949518100000001, 0.19993014, 0.194279588, 0.341892625, 0.39522561300000003, 0.341044708], Any[0.00304501, 0.003038741, 0.006447459, 0.003869151, 0.003935083, 0.0038591709999999998, 0.004139645], Any[0.00304501, 0.003038741, 0.006447459, 0.003869151, 0.003935083, 0.0038591709999999998, 0.004139645], 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, ["src-5-sink", "src-16-sink", "src-16-5-sink", "src-2-sink", "src-26-sink", "src-22-sink", "src-28-sink", "src-19-sink", "src-4-sink", "src-25-sink"  …  "src-29-5-22-20-28-19-12-1

# Comparison

Now we will compare the performance of each method.

In [ ]:
# tools to record solutions 
function record_outcome(outputs, file_name)
    lbs, ubs, mips, MP_times, SP_max_times, SP_times, x_val, _, vals, is_speedy = outputs
    serial_times = cumsum(MP_times .+ SP_times)
    parallel_times = cumsum(MP_times .+ SP_max_times)
    log = DataFrame(serial_time=serial_times,
                        parallel_time=parallel_times,
                        lower_bound=lbs,
                        upper_bound=ubs,
                        mip_bound=mips,
                        MP_time=MP_times,
                        SP_time=SP_times,
                        SP_max_time=SP_max_times,
                        speedy=is_speedy)
    CSV.write(file_name, log)
    return log
end

function solve_method(
        D:: ItineraryPlanning.DayBefore,
        algorithm:: String, 
        speed:: String, 
        filename:: String, 
        swap:: Bool = true
    )
    if algorithm == "direct"
        solve_time_sec, _, _, _, _ =  direct_model_solve(D, swap, !relax)
        return solve_time_sec
    end
    activated = (algorithm == "ABD")
    if speed == "PO"
        pargs = regular_po
    elseif speed == "LPO"
        pargs = local_po 
    elseif speed == "ALPO"
        pargs = act_local_po 
    else 
        pargs = NULLPARETO 
    end
    outputs = benders(activated, D, swap, act_local_po)
    if length(outputs) > 0 
        log = record_outcome(outputs, filename)
        return log[end, "serial_time"]
    end
    return -1
end 

In [ ]:
# Appendix C.1 Table 9
alg_options = ["direct", "BD", "ABD"]
accel_options = ["none", "PO", "LPO", "ALPO"]
solve_times = Dict()
for num_scen in [1, 25]
    D = DayBefore(data_dir, num_scen, num_itin, W, f, labor_min_planned, labor_min_adhoc, delay_penalty)
    for method in alg_options 
        if method == "direct"
            accel = "none"
            filepath = joinpath(output_dir, method * "_" * string(num_scen) * "scen.csv")
            solve_times[(num_scen, method, accel)] = solve_method(D, method, accel, filepath)
        else 
            for accel in accel_options
                filepath = joinpath(output_dir, method * "_" * accel * "_" * string(num_scen) * "scen.csv")
                solve_times[(num_scen, method, accel)] = solve_method(D, method, accel, filepath)    
            end
        end
    end
end
alg_dat = DataFrame([(num_scen, method, accel, st) for ((num_scen, method, accel), st) in solve_times])
rename!(alg_dat, ["num_scen", "algorithm", "speed_method", "solve_time_sec"])
CSV.write(joinpath(output_dir, "summary_more.csv"), alg_dat)

Note that, if "speedy" is true, the lower bound is approximate--it will sometimes exceed the upper / MIP bounds, which are true.